In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2024-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-30,1,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-02-01,1,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2024-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81350,9.0,6.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,2.0,...,1.0,2.0,6.0,1.0,1.0,0.0,1.0,2024-11-26,265,1.0
81351,10.0,3.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,2.0,3.0,6.0,1.0,2024-11-27,265,0.0
81352,6.0,7.0,7.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,...,2.0,5.0,4.0,2.0,0.0,2.0,6.0,2024-11-28,265,0.0
81353,16.0,3.0,5.0,5.0,2.0,1.0,4.0,2.0,2.0,0.0,...,7.0,0.0,2.0,3.0,1.0,1.0,2.0,2024-11-29,265,2.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2024, 7, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(40810, 674)
y_train.shape=(40810,)
X_test.shape=(40545, 674)
y_test.shape=(40545,)


In [4]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()


In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2025-02-13 22:32:52,851] A new study created in memory with name: no-name-9d79fff7-88d7-4c14-9108-8fa48785ea36
[I 2025-02-13 22:33:21,289] Trial 0 finished with value: 3.0474019563142 and parameters: {'num_leaves': 169, 'feature_fraction': 0.43506525316222266, 'bagging_fraction': 0.36927059147002184, 'min_child_samples': 57}. Best is trial 0 with value: 3.0474019563142.
[I 2025-02-13 22:34:05,624] Trial 1 finished with value: 3.144985517420689 and parameters: {'num_leaves': 168, 'feature_fraction': 0.6066102672885596, 'bagging_fraction': 0.37787401729368214, 'min_child_samples': 92}. Best is trial 0 with value: 3.0474019563142.
[I 2025-02-13 22:34:29,363] Trial 2 finished with value: 3.1243808177094796 and parameters: {'num_leaves': 49, 'feature_fraction': 0.46036489874372377, 'bagging_fraction': 0.406805214374782, 'min_child_samples': 6}. Best is trial 0 with value: 3.0474019563142.
[I 2025-02-13 22:35:14,365] Trial 3 finished with value: 2.949270152665178 and parameters: {'num_lea

In [6]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 126, 'feature_fraction': 0.7767096121209607, 'bagging_fraction': 0.3628931220432577, 'min_child_samples': 63}


In [7]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001A7C35B5620>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.3628931220432577,
                               feature_fraction=0.7767096121209607,
                               min_child_samples=63, num_leaves=126))])

In [8]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=3.0319


In [9]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [10]:
plot_one_sample(
    example_id=1300,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)